### Instructions

Credit reports are part of the external data recovered by FairMoney. The point here is to evaluate your ability to process semi-structured data to extract features that you think are relevant for credit scoring.

- From the file *credit_report_sample.json*, the goal is to extract/create all the features that seem relevant to you.
- For each constructed variable, tell us how it could be relevant to improve the risk scoring model.
- The output should be a Python function/class which takes one or more credit reports as input and returns the features as they can be used by a model.

We will evaluate your ability to manipulate semi-structured data and your business sense.

# Notebook Summary
A single function has been built to construct different variables neccessary for augmenting a risk model for improved performance.

This function takes the json file (or link in this case) and generates a ready to use dataframe containing diffent variables considered neccessary to augment a Credit Risk Model for FairMoney.


What to Expect?
There are two important section for review.
1. **The Code Block.** A detailed function designed to mung the JSON file and develop a variable list of 39 columns.
2. **Data Extraction and Field Selection:** Containing a documentation of sections of the JSON and fields extracted from these sections to develop the dataset. It also details each field in the final dataframe.




# Code Block

In [ ]:

import pandas as pd
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta


def process_credit_data(json_link):
    """
    Fetches, extracts, and processes credit data from a JSON link.

    Args:
      json_link: The URL of the JSON data.

    Returns:
      A pandas DataFrame with the processed credit data.
    """

    # Example usage:
    json_link = "https://file.notion.so/f/f/ce90a377-5991-40c1-8a79-ada0a7d2355e/9f5d6694-7864-4e64-9e1b-d65c348946dc/credit_report_sample.json?table=block&id=9f3e46d5-f284-4b6a-9df5-7a753399db78&spaceId=ce90a377-5991-40c1-8a79-ada0a7d2355e&expirationTimestamp=1728849600000&signature=3jz4si6bwMG-bPVqruqBMco9yuq3eoUd_SRz8DmaAfQ&downloadName=credit_report_sample.json"  # Replace with the actual link to your JSON data

    response = requests.get(json_link)
    data = response.json()

    # --- Extract data into DataFrames ---

    # Account Rating
    account_rating_data = []
    for item in data:
        account_rating = item['data']['consumerfullcredit']['accountrating']
        account_rating['application_id'] = item['application_id']
        account_rating_data.append(account_rating)
    df_accountrating = pd.DataFrame(account_rating_data)

    # Credit Account Summary
    credit_account_summary_data = []
    for item in data:
        credit_account_summary = item['data']['consumerfullcredit']['creditaccountsummary']
        credit_account_summary['application_id'] = item['application_id']
        credit_account_summary_data.append(credit_account_summary)
    df_credit_summary = pd.DataFrame(credit_account_summary_data)[
        ['rating', 'amountarrear', 'totalaccounts', 'totalaccountarrear',
         'totaljudgementamount', 'totaloutstandingdebt', 'totaldishonouredamount',
         'totalmonthlyinstalment', 'totalnumberofjudgement', 'totalnumberofdishonoured',
         'totalaccountingodcondition', 'application_id']
    ]

    # Define the columns to convert to numeric data type
    columns_to_convert = ['amountarrear', 'totalaccounts', 'totalaccountarrear',
                          'totaljudgementamount', 'totaloutstandingdebt',
                          'totaldishonouredamount', 'totalmonthlyinstalment']

    # Convert the columns directly within the code
    for col in columns_to_convert:
        if col in df_credit_summary.columns:
            df_credit_summary[col] = pd.to_numeric(df_credit_summary[col].astype(str).str.replace(',', ''), errors='coerce')


    # Enquiry History
    enquiry_history_data = []
    for item in data:
        enquiry_history = item['data']['consumerfullcredit']['enquiryhistorytop']
        for enquiry in enquiry_history:
            enquiry['application_id'] = item['application_id']
            enquiry_history_data.append(enquiry)
    df_enquiry = pd.DataFrame(enquiry_history_data)

    # Personal Details
    personal_details_data = []
    for item in data:
        personal_details = item['data']['consumerfullcredit']['personaldetailssummary']
        personal_details['application_id'] = item['application_id']
        personal_details_data.append(personal_details)
    df_personal_details = pd.DataFrame(personal_details_data)[["gender", "birthdate", "application_id", "dependants"]]

    # Account Monthly Payment History
    account_monthly_payment_data = []
    for item in data:
        monthly_payments = item['data']['consumerfullcredit']['accountmonthlypaymenthistory']
        for payment in monthly_payments:
            payment['application_id'] = item['application_id']
            account_monthly_payment_data.append(payment)
    df_monthly_payments = pd.DataFrame(account_monthly_payment_data)

    # --- Data transformations ---

    # Convert 'birthdate' to datetime objects and calculate age
    df_personal_details['birthdate'] = pd.to_datetime(df_personal_details['birthdate'], format='%d/%m/%Y', errors='coerce').dt.date
    today = datetime.today().date()
    df_personal_details['age'] = df_personal_details['birthdate'].apply(lambda birthdate: relativedelta(today, birthdate).years)
    df_personal_details.drop(columns=['birthdate'], inplace=True)

    # Convert 'daterequested' to datetime objects
    df_enquiry['daterequested'] = pd.to_datetime(df_enquiry['daterequested'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

    # --- Feature engineering ---
    # Calculate payment consistency and aggregate by application_id
    df_monthly_payments['payment_consistency'] = df_monthly_payments.apply(calculate_payment_consistency, axis=1)
    avg_payment_consistency = df_monthly_payments.groupby('application_id')['payment_consistency'].mean().reset_index()

    # Group by 'application_id' and calculate the number of enquiries
    enquiry_counts = df_enquiry.groupby('application_id')['daterequested'].count().reset_index(name='num_enquiries')

    # Calculate the average days between enquiries for each application_id
    average_days_between_enquiries = df_enquiry.groupby('application_id').apply(
        calculate_average_days_between_enquiries).reset_index(name='avg_days_between_enquiries')

    # Merge the two results
    enquiry_stats = pd.merge(enquiry_counts, average_days_between_enquiries, on='application_id')


    # Extract creditagreementsummary data
    credit_agreements_data = []
    for item in data:
        credit_agreements = item['data']['consumerfullcredit']['creditagreementsummary']
        for agreement in credit_agreements:  # Iterate through the list of agreements
            agreement['application_id'] = item['application_id']  # Include application_id
            credit_agreements_data.append(agreement)

    # Create the DataFrame
    df_credit_agreements = pd.DataFrame(credit_agreements_data)
    df_credit_agreements['currentbalanceamt'] = pd.to_numeric(df_credit_agreements['currentbalanceamt'].astype(str).str.replace(',', ''), errors='coerce')
    # Aggregate df_credit_agreements (example - you'll need to add more aggregations)
    total_debt_per_application = df_credit_agreements.groupby('application_id')['currentbalanceamt'].sum().reset_index(name='total_debt')

    # Merge aggregated dataframes and original dataframes on application_id
    merged_data = pd.merge(df_credit_summary, total_debt_per_application, on='application_id')
    merged_data = pd.merge(merged_data, df_accountrating, on='application_id')
    merged_data = pd.merge(merged_data, df_personal_details, on='application_id')
    merged_data = pd.merge(merged_data, enquiry_stats, on='application_id')
    merged_data = pd.merge(merged_data, avg_payment_consistency, on='application_id')
    merged_data = sum_good_bad_columns_by_application(merged_data)

    return merged_data

# --- Helper functions ---

def calculate_payment_consistency(row):
    """Calculates payment consistency score from m01 to m24 fields."""

    weights = {
        "#": 0,
        "0": 1,
        "1": 0.8,
        "2": 0.6,
        "3": 0.4,
        "4": 0.2,
        "5": 0,  # You might need to adjust these weights
        # ... add weights for other codes as needed
    }

    total_weight = 0
    num_payments = 0
    for i in range(1, 25):  # Iterate through m01 to m24
        field = f"m{i:02d}"  # Create field name (e.g., "m01", "m02")
        code = row[field]
        if code in weights:
            total_weight += weights[code]
            num_payments += 1

    if num_payments > 0:
        return total_weight / num_payments
    else:
        return None  # Or handle cases with no payment history

def calculate_average_days_between_enquiries(group):
  if len(group) <= 1:
    return 0  # If only one enquiry, there's no interval

  sorted_dates = sorted(group['daterequested'].dropna())
  date_diffs = [(sorted_dates[i+1] - sorted_dates[i]).days for i in range(len(sorted_dates)-1)]
  return sum(date_diffs) / len(date_diffs)

def sum_good_bad_columns_by_application(df):
    """
    For each application_id, sums all columns with 'good' and 'bad'
    in their names after converting them to numerical types.
    Adds 'good_loans' and 'bad_loans' columns and drops original
    'good' and 'bad' columns.

    Args:
        df: The pandas DataFrame to process.

    Returns:
        A pandas DataFrame with 'good_loans' and 'bad_loans' columns.
    """

    df_copy = df.copy()

    good_columns = [col for col in df.columns if 'good' in col.lower()]
    bad_columns = [col for col in df.columns if 'bad' in col.lower()]

    for col in good_columns + bad_columns:
        df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')

    df_copy['good_loans'] = df_copy[good_columns].sum(axis=1)
    df_copy['bad_loans'] = df_copy[bad_columns].sum(axis=1)

   # df_copy = df_copy.drop(columns=good_columns + bad_columns)

    return df_copy

# --- Run the function ---

# Replace with your actual JSON link
# Example usage:
json_link = "https://file.notion.so/f/f/ce90a377-5991-40c1-8a79-ada0a7d2355e/9f5d6694-7864-4e64-9e1b-d65c348946dc/credit_report_sample.json?table=block&id=9f3e46d5-f284-4b6a-9df5-7a753399db78&spaceId=ce90a377-5991-40c1-8a79-ada0a7d2355e&expirationTimestamp=1728849600000&signature=3jz4si6bwMG-bPVqruqBMco9yuq3eoUd_SRz8DmaAfQ&downloadName=credit_report_sample.json"  # Replace with the actual link to your JSON data

final_df = process_credit_data(json_link)
final_df

<ipython-input-5-5745a2b02136>:104: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  average_days_between_enquiries = df_enquiry.groupby('application_id').apply(


,rating,amountarrear,totalaccounts,totalaccountarrear,totaljudgementamount,totaloutstandingdebt,totaldishonouredamount,totalmonthlyinstalment,totalnumberofjudgement,totalnumberofdishonoured,...,noofpersonalloanaccountsbad,noofpersonalloanaccountsgood,gender,dependants,age,num_enquiries,avg_days_between_enquiries,payment_consistency,good_loans,bad_loans
0,13,24041.0,7,2,0,105435.0,0.0,77404.0,0,0,...,0,1,Male,0,33,15,13.357143,0.256429,7,0
1,2,0.0,17,1,0,294770.0,0.0,132176.0,0,0,...,0,2,Female,0,38,10,98.333333,0.124314,17,0
2,109,12000.0,3,1,0,110919.0,0.0,7000.0,0,0,...,0,0,Female,0,41,10,30.555556,0.325000,2,1


In [ ]:
#write final_df to excel and save to drive

from google.colab import drive
drive.mount('/content/drive')

# Specify the path where you want to save the Excel file in your Google Drive
excel_file_path = '/content/drive/My Drive/credit_data.xlsx'

# Save the DataFrame to an Excel file
final_df.to_excel(excel_file_path, index=False)  # Set index=False to avoid saving the index


Mounted at /content/drive


# Data Extraction and Field Selection:

This code extracts and processes credit information from a JSON link to build a risk model. Here's a breakdown of the key data used:

* **Account Rating:** Summarizes the types of credit accounts (e.g., auto, home, personal loans) and their performance (good vs. bad). This helps assess the borrower's credit mix and identify potential risk areas.

* **Credit Account Summary:** Provides an overview of the borrower's credit health, including their credit rating, outstanding debt, and delinquency status. Key fields include rating, amountarrear, totalaccounts, totaloutstandingdebt, etc.

* **Enquiry History:** Tracks inquiries made about the borrower's credit report (enquiryhistorytop). Frequent inquiries can signal financial stress.

* **Personal Details:**  Includes basic demographics like gender, birthdate, and dependants, which can provide additional context for risk assessment.

* **Account Monthly Payment History:** Contains detailed payment history (m01 to m24) for each account. This data is used to engineer a **payment_consistency** score, which quantifies how reliably the borrower makes payments. This score, ranging from 0 to 1, is calculated by assigning weights to the payment codes in the m01 to m24 fields and calculating a weighted average. A higher score indicates more consistent on-time payments and generally suggests lower credit risk.

## **Final Fields:**
**Credit Summary and Account Information**

1. rating: Overall credit score or risk rating. (Higher is generally better)
2. amountarrear: Total amount overdue on accounts. (Indicates current delinquency)
3. totalaccounts: Total number of credit accounts. (Reflects credit history length and potential utilization)
4. totalaccountarrear: Total number of accounts in arrears. (Shows breadth of delinquency)
5. totaljudgementamount: Total amount of judgments against the applicant. (Serious negative indicator)
6. totaloutstandingdebt: Total outstanding debt across all accounts. (Measures overall debt burden)
7. totaldishonouredamount: Total amount of dishonored payments. (Reflects payment reliability)
8. totalmonthlyinstalment: Total monthly installment payments. (Indicates monthly payment obligations)
9. totalnumberofjudgement: Total number of judgments. (Negative legal event)
10. totalnumberofdishonoured: Total number of dishonored payments. (Shows payment difficulties)
11. totalaccountingodcondition: Total number of accounts in overdrawn or bad standing. (Indicates account management issues)
12. total_debt: Aggregated total debt from creditagreementsummary. (More detailed debt information)

**Account Rating Breakdown**

13. noofotheraccountsbad: Number of "bad" other accounts. (Specific risk in miscellaneous accounts)
14. noofotheraccountsgood: Number of "good" other accounts.
15. noofretailaccountsbad: Number of "bad" retail accounts. (Risk in retail credit)
16. noofretailaccountsgood: Number of "good" retail accounts.
17. nooftelecomaccountsbad: Number of "bad" telecom accounts. (Risk in telecom credit)
18. noofautoloanaccountsbad: Number of "bad" auto loans. (Risk in auto loans)
19. noofautoloanccountsgood: Number of "good" auto loans.
20. noofhomeloanaccountsbad: Number of "bad" home loans. (Risk in mortgages)
21. noofhomeloanaccountsgood: Number of "good" home loans.
22. noofjointloanaccountsbad: Number of "bad" joint loans. (Risk in shared loans)
23. noofjointloanaccountsgood: Number of "good" joint loans.
24. noofstudyloanaccountsbad: Number of "bad" study loans. (Risk in student loans)
25. noofstudyloanaccountsgood: Number of "good" study loans.
26. noofcreditcardaccountsbad: Number of "bad" credit card accounts. (Risk in credit card debt)
27. noofcreditcardaccountsgood: Number of "good" credit card accounts.
28. noofpersonalloanaccountsbad: Number of "bad" personal loans. (Risk in personal loans)
29. noofpersonalloanaccountsgood: Number of "good" personal loans.
30. nooftelecomaccountsgood: Number of "good" telecom accounts. (Risk in telecom credit)

**Personal Details**

31. gender: Borrower's gender. (Can be a factor in risk, use cautiously)
32. dependants: Number of dependents. (May indirectly influence financial stability)
33. age: Borrower's age. (Can be a risk factor, use responsibly)

**Enquiry History**

34. num_enquiries: Number of credit inquiries. (High frequency can indicate risk)
35. avg_days_between_enquiries: Average days between inquiries. (Shows pattern of credit seeking)

**Engineered Features**

36. payment_consistency: Measure of consistent on-time payments. (Higher score is generally better)
37. good_loans: (This seems to be a new aggregated field - likely total good loans)
38. bad_loans: (This seems to be a new aggregated field - likely total bad loans)
39. Application_id: Unique Identified assigned to each credit application, which also reconciles to internal consumer_id


This detailed breakdown should give you a good understanding of each feature and its potential relevance to a credit risk model.